Use censusdata api to query unemployment rate and education levels at the zipcode level for all zipcodes in California

In [ ]:
import pandas as pd
import censusdata
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2)

In [ ]:
censusdata.search('acs5', 2015, 'label', 'unemploy')

In [ ]:
censusdata.search('acs5', 2012, 'concept', 'education')

In [ ]:
cookbg = censusdata.download('acs5', 2012,
                             censusdata.censusgeo([('zip code tabulation area', '*')]),
                             ['B23025_003E', 'B23025_005E', 'B15003_001E', 'B15003_002E', 'B15003_003E',
                              'B15003_004E', 'B15003_005E', 'B15003_006E', 'B15003_007E', 'B15003_008E',
                              'B15003_009E', 'B15003_010E', 'B15003_011E', 'B15003_012E', 'B15003_013E',
                              'B15003_014E', 'B15003_015E', 'B15003_016E'])

In [ ]:
cookbg.head()

In [ ]:
cookbg['percent_unemployed'] = cookbg.B23025_005E / cookbg.B23025_003E * 100
cookbg['percent_nohs'] = (cookbg.B15003_002E + cookbg.B15003_003E + cookbg.B15003_004E
                          + cookbg.B15003_005E + cookbg.B15003_006E + cookbg.B15003_007E + cookbg.B15003_008E
                          + cookbg.B15003_009E + cookbg.B15003_010E + cookbg.B15003_011E + cookbg.B15003_012E
                          + cookbg.B15003_013E + cookbg.B15003_014E +
                          cookbg.B15003_015E + cookbg.B15003_016E) / cookbg.B15003_001E * 100
cookbg = cookbg[['percent_unemployed', 'percent_nohs']]
cookbg.describe()

In [ ]:
cookbg['index1'] = cookbg.index

In [ ]:
cookbg1 = cookbg['index1'].astype(str)

In [ ]:
cookbg2= cookbg1.str.split(":", n = 1, expand = True)  
cookbg3= cookbg2[0].str.split(" ", n = 1, expand = True)
cookbg4 = cookbg3[1]
cookbg5 = cookbg4.rename(columns={0: "ZIPCODE"})

In [ ]:
cookbg6 = cookbg5.to_frame("Zipcode")

In [ ]:
zip_data = pd.merge(cookbg, cookbg6, left_index=True, right_index=True)

In [ ]:
zip_data = zip_data[['percent_unemployed','percent_nohs', 'Zipcode']]

In [ ]:
zip_data['zipnum'] = zip_data['Zipcode'].astype(int)

In [ ]:
zip_data = zip_data.loc[zip_data['zipnum'] > 89999]
zip_data = zip_data.loc[zip_data['zipnum'] < 100000]

In [ ]:
zip_data = zip_data.reset_index(drop=True)

In [ ]:
zip_data = zip_data[['percent_unemployed','percent_nohs', 'zipnum']]

In [ ]:
zip_data = zip_data.rename(columns={"zipnum": "zipcode"})

In [ ]:
zip_data.head()

In [ ]:
zip_data.to_pickle("Data/ACS/ACS_data.pkl")